# packet

> TODO fill in description

In [ ]:
#| default_exp packet

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
import asyncio
from typing import Any, Union
from enum import Enum

import fbdev
from fbdev.utils import AddressableMixin

In [ ]:
#|hide
show_doc(fbdev.packet.NullPayload)

---

### NullPayload

>      NullPayload ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class NullPayload:
    _instance = None
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(NullPayload, cls).__new__(cls)
        return cls._instance
    def __repr__(self):
        return "<NullPayload>"

In [ ]:
#|hide
show_doc(fbdev.packet.Packet)

---

### Packet

>      Packet (_data:Any)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class Packet:
    def __init__(self, *, _data:Any):
        self._data = _data
        self._dtype = type(_data)
            
    @property
    def is_empty(self): return self._dtype == NullPayload
    @property
    def dtype(self): return self._dtype
    
    def _move(self, address):
        pass
    
    def get_deep_copy(self):
        pass

In [ ]:
#|hide
show_doc(fbdev.packet.PacketRegistry)

---

### PacketRegistry

>      PacketRegistry ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class PacketRegistry:
    def __init__(self): 
        self._packets = set()
        self._consumed_packets = set()
        self._packet_locations = {}
        self._packet_history = {}
        
    def get_location(self, packet:Packet):
        if not self.is_registered(packet): raise ValueError("Packet not in registry.")
        return self._packet_locations[packet]
    
    def is_consumed(self, packet:Packet):
        return packet in self._consumed_packets
    
    def is_registered(self, packet:Packet):
        return packet in self._packets
        
    def create(self, data:Any, address):
        if type(data) == Packet:
            raise ValueError("Cannot create packet from another packet.")
        packet = Packet(_data=data)
        self._packets.add(packet)
        self._packet_locations[packet] = address
        self._packet_history[packet] = []
        return packet
    
    def create_empty(self, address):
        return self.create(NullPayload(), address)
        
    def register_move(self, packet:Packet, dest:AddressableMixin, port_type:fbdev.port.PortType, port_name:str):
        if not self.is_registered(packet): raise ValueError("Packet not in registry.")
        if self.is_consumed(packet): raise ValueError("Packet is consumed.")
        current_address = self.get_location(packet)
        if dest.address == current_address: raise ValueError("Packet is already at destination.")
        dest_address = dest.address
        moving_via = (port_type, port_name)
        self._packet_history[packet].append((current_address, dest_address, moving_via))
        self._packet_locations[packet] = dest_address
        
    async def consume(self, packet:Packet):
        if not self.is_registered(packet): raise ValueError("Packet not in registry.")
        if self.is_consumed(packet): raise ValueError("Packet already consumed.")
        del self._packet_locations[packet]
        self._packets.remove(packet)
        self._consumed_packets.add(packet)
        data = packet._data # TODO this will be changed when we also have remote packets
        packet._data = None
        return data
    
    async def peek(self, packet:Packet):
        data = self._data # TODO this will be changed when we also have remote packets
        return data

In [ ]:
#|hide
show_doc(fbdev.packet.PacketHandler)

---

### PacketHandler

>      PacketHandler
>                     (registry:Union[fbdev.packet.PacketRegistry,fbdev.packet.P
>                     acketHandler], parent_node:fbdev.node.Node)

*A handler for packets for Nodes. The handler knows its own address, and verifies the packets against it.*

In [ ]:
#|export
class PacketHandler:
    """
    A handler for packets for Nodes. The handler knows its own address, and verifies the packets against it.
    """
    def __init__(self, registry:Union[PacketRegistry, PacketHandler], parent_node:fbdev.node.Node):
        if isinstance(registry, PacketHandler):
            self._registry = registry._registry
        else:
            self._registry = registry
        self._parent_node = parent_node
        
    def _verify_location(self, packet:Packet):
        if self._registry.get_location(packet) != self._parent_node.address:
            raise ValueError(f"Packet location is {self._registry.get_location(packet)}, but expected {self._parent_node.address}.")
        
    def is_consumed(self, packet:Packet):
        return self._registry.is_consumed(packet)
        
    def create(self, data:Any):
        return self._registry.create(data, self._parent_node.address)
    
    def create_empty(self):
        return self._registry.create_empty()
    
    def register_move(self, packet:Packet, dest:AddressableMixin, port_type:fbdev.port.PortType, port_name:str):
        if dest.address != self._parent_node.address:
            self._verify_location(packet) # If the packet is being sent away, then verify that it is currently located at self._parent_node
        self._registry.register_move(packet, dest, port_type, port_name)
        
    async def consume(self, packet:Packet):
        self._verify_location(packet)
        return await self._registry.consume(packet)
    
    async def peek(self, packet:Packet):
        self._verify_location(packet)
        return await self._registry.peek(packet)
    
    def get_location(self, packet:Packet):
        return self._registry.get_location(packet)